In [1]:
import os
import numpy as np
import pandas as pd
import time
import scanpy as sc
import torch

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)

import sys
sys.path.append('../../')
import scDualGN
sys.path.append('../')
import expri_helper

/jdfssz1/ST_HEALTH/P18Z10200N0124/AI/user/baiyong/anaconda3/envs/mytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dat_path = expri_helper.get_path('Reyes', dat_ls_file='../expri_data.json')

In [3]:
#load data
adata = sc.read_h5ad(dat_path)
adata.obs['celltype'] = adata.obs['cell_type']
adata.obs['celltype_str'] = adata.obs['cell_type']

cell_type_dict = {'T':0, 'B':1, 'NK':2, 'Mono':3, 'DC':4} 
adata.obs['celltype'] = adata.obs['celltype'].map(cell_type_dict)

# arr = np.array(adata.obs['celltype'])
# np.place(arr,arr=='T',[0])
# np.place(arr,arr=='B',[1])
# np.place(arr,arr=='NK',[2])
# np.place(arr,arr=='Mono',[3])
# np.place(arr,arr=='DC',[4])


# arr = arr.astype(np.int32)
# adata.obs['celltype'] = arr

In [4]:
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2500)
adata = adata[:, adata.var.highly_variable]
    #adata.X = minmax_scale(adata.X, feature_range=(0, 1), axis=1, copy=False)
sc.pp.scale(adata,max_value=10)

In [5]:
#kmeans
# acc=0.9019, nmi=0.7987, ari=0.8511
# acc 0.8319, nmi 0.6950, ari 0.7245, hs 0.6274, cs 0.7789, purity: 0.8457
start = time.time()
# z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata, n_cluster=5,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.025,beta_daulvae=20,gamma_dualvae=2,GPU_id=1)

# acc 0.8272, nmi 0.6740, ari 0.7173, hs 0.6093, cs 0.7540, purity: 0.8428 batch_size=512, lr=0.001


# scdualgn_model = scDualGN.scDualGN(adata, n_z=32,device='cuda:2', verbosity=True, batch_size=1024*10).pretrain(alpha=0.05, beta=20, gamma=4)
# scdualgn_model.cluster(n_cluster=5, eta=1, nu=0.1, n_epochs=30)

lr, alpha, gamma, nu = 0.01, 0.01, 4, 0.01
print('=====================================')
print('lr:{}, alpha:{}, gamma:{}, nu:{}'.format(lr, alpha, gamma, nu))
start = time.time()
torch.cuda.empty_cache()
scdualgn_model = scDualGN.scDualGN(adata, n_z=32,device='cuda:1', batch_size=1024*8, verbosity=True, lr=lr).pretrain(alpha=alpha, beta=1, gamma=gamma)
scdualgn_model.cluster(n_cluster=5, eta=1, nu=nu, n_epochs=30)

end = time.time()
print('running time = {}'.format(end-start))


INFO:scDualGN.run:dual-VAE pretrain start...


lr:0.01, alpha:0.01, gamma:4, nu:0.01


INFO:scDualGN.run:Epoch 1/50,Overall loss:899.2846,MSE:711.3660,MSE1:45.8454,KL: 453.6833
INFO:scDualGN.run:Epoch 2/50,Overall loss:582.0811,MSE:577.0937,MSE1:0.1274,KL: 447.7970
INFO:scDualGN.run:Epoch 3/50,Overall loss:575.3424,MSE:571.5496,MSE1:0.1384,KL: 323.9254
INFO:scDualGN.run:Epoch 4/50,Overall loss:570.7040,MSE:567.6257,MSE1:0.1387,KL: 252.3731
INFO:scDualGN.run:Epoch 5/50,Overall loss:566.4607,MSE:563.7979,MSE1:0.1406,KL: 210.0189
INFO:scDualGN.run:Epoch 6/50,Overall loss:563.3849,MSE:560.9553,MSE1:0.1427,KL: 185.8863
INFO:scDualGN.run:Epoch 7/50,Overall loss:560.6461,MSE:558.3482,MSE1:0.1418,KL: 173.0623
INFO:scDualGN.run:Epoch 8/50,Overall loss:558.6444,MSE:556.4227,MSE1:0.1402,KL: 166.0906
INFO:scDualGN.run:Epoch 9/50,Overall loss:557.2219,MSE:555.0394,MSE1:0.1393,KL: 162.5446
INFO:scDualGN.run:Epoch 10/50,Overall loss:556.4171,MSE:554.2519,MSE1:0.1386,KL: 161.1001
INFO:scDualGN.run:Epoch 11/50,Overall loss:566.2531,MSE:564.1449,MSE1:0.1419,KL: 154.0787
INFO:scDualGN.run:

acc 0.8469, nmi 0.6785, ari 0.7868, hs 0.6673, cs 0.6900, purity: 0.8469
running time = 668.2432432174683


In [6]:
#leiden
# start = time.time()
# z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.025,beta_daulvae=20,gamma_dualvae=2,GPU_id=1,
#                                                    cluster_alg='leiden',n_neighbors=50)
# end = time.time()

# print('running time = {}'.format(end-start))